# Phase 3a: Ordinal Logistic Regression (Deep Dive)

Mục tiêu: Xây dựng và phân tích mô hình hồi quy thứ bậc để hiểu rõ tác động của từng nhóm thực phẩm lên điểm số NRF9.3 (Bad -> Medium -> Good).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 1. Load Data
df = pd.read_csv('labeled_data_final.csv')
print(f"Data Shape: {df.shape}")

In [ ]:
# 2. Map Target
label_map = {'Bad': 0, 'Medium': 1, 'Good': 2}
df['y'] = df['Label_Text'].map(label_map)

## Bước 1: Feature Engineering (Chuẩn hóa năng lượng)
Vì NRF9.3 tính trên 100kcal, nên Features (FPED) cũng phải được quy đổi về cùng hệ quy chiếu (per 100kcal).

In [ ]:
print("Đang tạo các biến Scaled per 100kcal...")
# 1. Xác định các cột FPED cần chuẩn hóa
# Lấy tất cả cột trừ ID, Description, Energy, và các cột kết quả
# CẬP NHẬT: Loại bỏ thêm Total_NRF9 và Total_LIM vì đây là biến trung gian tính ra kết quả (Target Leakage)
exclude_cols = ['ID', 'Energy (kcal)', 'Energy_Factor', 'NRF9.3_Score', 'Label_Value', 'y', 'WWEIA Category number', 
                'Total_NRF9', 'Total_LIM']

# Lọc lấy các cột FPED (thường có đơn vị cup eq. hoặc oz. eq.)
fped_cols = [c for c in df.columns if c not in exclude_cols and df[c].dtype in ['float64', 'int64']]
# Loại bỏ thêm các cột dinh dưỡng (Nutrients) nếu có
exclude_keywords = ['Protein', 'Fiber', 'Vitamin', 'Calcium', 'Iron', 'Magnesium', 'Potassium', 'Sodium', 'Fatty', 'Sugar', 'Label_Text', 'DESCRIPTION']
fped_cols = [c for c in fped_cols if not any(k in c for k in exclude_keywords)]

# 2. Thực hiện chuẩn hóa
# Tính mẫu số năng lượng (Sàn 5 kcal)
energy_denom = df['Energy (kcal)'].clip(lower=5)
scaling_factor = 100 / energy_denom

scaled_features = []
for col in fped_cols:
    new_col_name = col + '_100kcal'
    df[new_col_name] = df[col] * scaling_factor
    scaled_features.append(new_col_name)

print(f"Đã tạo {len(scaled_features)} features chuẩn hóa.")

## Bước 2: Chọn Features & Kiểm tra VIF
Loại bỏ các cột 'TOTAL' để tránh đa cộng tuyến.

In [ ]:
# 3. Định nghĩa Features (X) - LỌC BỎ BIẾN TỔNG
# Danh sách 'Total' cần loại bỏ (phiên bản đã scale)
drop_totals = [
    'F_TOTAL (cup eq.)_100kcal', 'V_TOTAL (cup eq.)_100kcal', 'V_REDOR_TOTAL (cup eq.)_100kcal', 
    'V_STARCHY_TOTAL (cup eq.)_100kcal', 'G_TOTAL (oz. eq.)_100kcal', 'PF_TOTAL (oz. eq.)_100kcal', 
    'PF_MPS_TOTAL (oz. eq.)_100kcal', 'D_TOTAL (cup eq.)_100kcal',
    'PF_LEGUMES (oz. eq.)_100kcal' # High VIF (>6000) duplicate with V_LEGUMES
]

# X chỉ lấy các cột trong danh sách scaled_features và KHÔNG nằm trong drop_totals
final_features = [c for c in scaled_features if c not in drop_totals]

X = df[final_features]
y = df['y']

print(f"Số lượng Features cuối cùng: {X.shape[1]}")

# 4. Kiểm tra VIF (Variance Inflation Factor)
print("\n--- KIỂM TRA ĐA CỘNG TUYẾN (VIF) ---")
X_vif = X.copy()
X_vif['const'] = 1
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_vif.columns))]

print(vif_data.sort_values('VIF', ascending=False).head(10))
print("Lưu ý: Nếu VIF < 10 là tốt.")

## Bước 3: Train Model (OrderedModel)

In [ ]:
# 5. Chia tập Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 6. Huấn luyện (OrderedModel)
model = OrderedModel(y_train, X_train, distr='logit')
res = model.fit(method='bfgs', maxiter=1000)

# Hiển thị bảng kết quả thống kê
print(res.summary())

## Bước 4: Phân tích Odds Ratio

In [ ]:
# 7. Tính Odds Ratio và lọc P-value < 0.05
params = res.params
conf = res.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['CI Lower', 'CI Upper', 'Odds Ratio']
conf = np.exp(conf)

print("\n--- CÁC YẾU TỐ TÁC ĐỘNG MẠNH NHẤT (P < 0.05) ---")
significant_vars = res.pvalues[res.pvalues < 0.05].index
significant_res = conf.loc[significant_vars]
print(significant_res.sort_values('Odds Ratio', ascending=False))

# Đánh giá
predicted = res.model.predict(res.params, X_test)
y_pred = predicted.argmax(1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Bad', 'Medium', 'Good']))

## Bước 5: Kiểm tra Giả định Proportional Odds
Ordinal Regression giả định rằng hệ số tác động là giống nhau giữa các ngưỡng cắt (Bad vs >Bad) và (<Good vs Good).
Nếu hệ số khác nhau quá nhiều, giả định bị vi phạm.

In [ ]:
from sklearn.linear_model import LogisticRegression

print("\n--- KIỂM TRA GIẢ ĐỊNH PROPORTIONAL ODDS ---")

# 1. Chạy 2 mô hình Logistic nhị phân riêng biệt
# Cắt 1: Bad (0) vs [Medium, Good] (1)
y_cut1 = (y_train > 0).astype(int)
model_cut1 = LogisticRegression(solver='lbfgs', max_iter=1000)
model_cut1.fit(X_train, y_cut1)

# Cắt 2: [Bad, Medium] (0) vs Good (1)
y_cut2 = (y_train > 1).astype(int)
model_cut2 = LogisticRegression(solver='lbfgs', max_iter=1000)
model_cut2.fit(X_train, y_cut2)

# 2. So sánh hệ số
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coef_Cut1 (Bad vs Above)': model_cut1.coef_[0],
    'Coef_Cut2 (Below vs Good)': model_cut2.coef_[0]
})

coef_df['Diff'] = (coef_df['Coef_Cut1 (Bad vs Above)'] - coef_df['Coef_Cut2 (Below vs Good)']).abs()
coef_df = coef_df.sort_values(by='Diff', ascending=False)

print("Top 5 biến vi phạm giả định nhất (Hệ số thay đổi nhiều nhất giữa các mức):")
print(coef_df.head(5))

# 3. Vẽ biểu đồ so sánh
plt.figure(figsize=(8, 8))
plt.scatter(coef_df['Coef_Cut1 (Bad vs Above)'], coef_df['Coef_Cut2 (Below vs Good)'], alpha=0.7)
plt.plot([-5, 5], [-5, 5], 'r--', label='Ideal Proportionality') # Đường chéo y=x
plt.xlabel('Hệ số (Bad vs Medium/Good)')
plt.ylabel('Hệ số (Bad/Medium vs Good)')
plt.title('Biểu đồ kiểm tra Proportional Odds')
plt.legend()
plt.grid(True)
plt.show()

## Bước 6: Dự báo cho Input bất kỳ (Manual Prediction)

In [ ]:
def predict_custom_food(model_res, input_data, feature_names, energy_kcal):
    """
    Dự báo nhãn cho một món ăn mới.
    input_data: Dictionary chứa các giá trị FPED GỐC (VD: {'F_CITMLB (cup eq.)': 1.0})
    energy_kcal: Năng lượng của món ăn (để scale)
    """
    # 0. Feature Engineering (Scale inputs)
    denom = max(energy_kcal, 5)
    factor = 100 / denom
    
    scaled_input = {}
    # Lặp qua các features mà mô hình cần (dạng _100kcal)
    for feat in feature_names:
        # Tên gốc tương ứng (bỏ đuôi _100kcal)
        orig_name = feat.replace('_100kcal', '')
        # Lấy giá trị từ input, nếu không có thì = 0
        raw_val = input_data.get(orig_name, 0.0)
        # Scale
        scaled_input[feat] = raw_val * factor
        
    # 1. Tạo DataFrame input
    input_df = pd.DataFrame([scaled_input])
    input_df = input_df[feature_names]
    
    # 2. Dự đoán
    predicted = model_res.model.predict(model_res.params, input_df)
    pred_idx = predicted.argmax(1)[0]
    labels = ['Bad', 'Medium', 'Good']
    result = labels[pred_idx]
    
    print(f"--- DỰ ĐOÁN KẾT QUẢ ---")
    print(f"Input Raw: {input_data}")
    print(f"Energy: {energy_kcal} kcal (Factor: {factor:.2f})")
    print(f"Xác suất Bad:    {predicted[0][0]:.2%}")
    print(f"Xác suất Medium: {predicted[0][1]:.2%}")
    print(f"Xác suất Good:   {predicted[0][2]:.2%}")
    print(f"==> KẾT LUẬN: {result}")
    return result

# --- TEST VÍ DỤ ---
# 1. Quả Cam (Tốt): Calo thấp, nhiều VitC (F_CITMLB)
test_good = {'F_CITMLB (cup eq.)': 1.0}
predict_custom_food(res, test_good, X.columns, 45)

# 2. Nước ngọt (Xấu): Calo cao (đường), dinh dưỡng rỗng
# Lưu ý: Mặc dù ta bỏ Added Sugar ra khỏi feature, nhưng G_REFINED thường đi kèm
# Hoặc chỉ đơn giản là thiếu các chất khác.
test_bad = {'G_REFINED (oz. eq.)': 0.0} # Nhập rỗng thử xem sao
predict_custom_food(res, test_bad, X.columns, 150)
